In [61]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as Dataset
import torchvision.transforms.functional as TVF
import torch.autograd as autograd
from torch.utils.data import Dataset, DataLoader

from torchvision.models import vgg16 , inception_v3 , resnet50
from PIL import Image

import nltk
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

import psutil
import time

In [9]:
t1 = torch.zeros(3,4)
t1.cuda()

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [10]:
psutil.virtual_memory()

svmem(total=17046654976, available=9272930304, percent=45.6, used=7773724672, free=9272930304)

In [11]:
columns = ['filename', 'caption']
caption_data = pd.read_csv('./Flickr8k_text/Flickr8k_token.txt' , 
                           sep = '\t' ,
                          header = None,
                          names = columns)
caption_data.head()

,filename,caption
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [12]:
caption_data['caption_num'] = caption_data['filename'].str.split('#',expand=True)[1]
caption_data.head()

,filename,caption,caption_num
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,4


In [13]:
caption_data.shape

(40460, 3)

In [14]:
40460 / 5

8092.0

In [15]:
caption_data['word_count'] = caption_data['caption'].apply(lambda x: len(str(x).split(" ")))
caption_data['word_count'].describe(percentiles = [0.25 , 0.5 , 0.75 , 0.9 , 0.95 , 0.99])

count    40460.000000
mean        11.782155
std          3.885175
min          1.000000
25%          9.000000
50%         11.000000
75%         14.000000
90%         17.000000
95%         19.000000
99%         23.000000
max         38.000000
Name: word_count, dtype: float64

In [16]:
word_idx = 0
word_to_idx_dict = {}
idx_to_word_dict = {}
sentence_list = []
for i in caption_data['caption']:
    text = i.lower().split(' ')
    sentence_list.append(text)
    for j in text:
        if word_to_idx_dict.get(j,0) == 0:
            word_to_idx_dict[j] = word_idx
            idx_to_word_dict[word_idx] = j
            word_idx+=1
        else: 
            pass

In [17]:
print('word_to_idx_dict : {}'.format(len(word_to_idx_dict)))
print('idx_to_word_dict : {}'.format(len(idx_to_word_dict)))

word_to_idx_dict : 8918
idx_to_word_dict : 8919


In [18]:
idx_to_word_dict[8918]

'patterns'

In [19]:
word_to_idx_dict['patterns']

8918

In [20]:
#vgg16()

In [21]:
#model_vgg16 = vgg16(pretrained=True)
#model_vgg16

In [22]:
#model_vgg16.classifier = nn.Linear(in_features=25088, out_features=4096, bias=True)

In [39]:
p = torchvision.transforms.Compose([torchvision.transforms.Resize((299,299))])
image_dict = {}
ctr = 0
path =  'D:\\Data_Science\\image_captioning\\Flickr8k_Dataset\\Flicker8k_Dataset'
total_files = len(os.listdir(path))
st_time = time.time()
for file in os.listdir(path):
    image = Image.open(path + '\\' + file)
    image = p(image)
    image_dict[file] = TVF.to_tensor(image)
    ctr+=1
    if ctr % 100 == 0:
        print(' file : {} / {} in {} seconds'.format(ctr , total_files , (time.time() - st_time)))
        st_time = time.time()

 file : 100 / 8091 in 0.4971456527709961 seconds
 file : 200 / 8091 in 0.5387814044952393 seconds
 file : 300 / 8091 in 0.5374798774719238 seconds
 file : 400 / 8091 in 0.5716202259063721 seconds
 file : 500 / 8091 in 0.5245392322540283 seconds
 file : 600 / 8091 in 0.5654861927032471 seconds
 file : 700 / 8091 in 0.5299861431121826 seconds
 file : 800 / 8091 in 0.5591206550598145 seconds
 file : 900 / 8091 in 0.543346643447876 seconds
 file : 1000 / 8091 in 0.4929971694946289 seconds
 file : 1100 / 8091 in 0.5432205200195312 seconds
 file : 1200 / 8091 in 0.5275876522064209 seconds
 file : 1300 / 8091 in 0.511976957321167 seconds
 file : 1400 / 8091 in 0.5286915302276611 seconds
 file : 1500 / 8091 in 0.5104763507843018 seconds
 file : 1600 / 8091 in 0.5081908702850342 seconds
 file : 1700 / 8091 in 0.5837013721466064 seconds
 file : 1800 / 8091 in 0.5308177471160889 seconds
 file : 1900 / 8091 in 0.5218682289123535 seconds
 file : 2000 / 8091 in 0.5543076992034912 seconds
 file : 210

In [40]:
seq_length = 38
vocab_size = len(word_to_idx_dict)
embed_size = 4096
hidden_size = embed_size

In [56]:
image.shape

torch.Size([1, 3, 299, 299])

In [66]:
ic = resnet50(pretrained=True)
ic.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [64]:
out = ic(image)
out.shape

torch.Size([1, 1000])

In [67]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        
        # get the pretrained vgg16 model
        #self.vgg16 = vgg16(pretrained=True)
        self.resnet50 = resnet50(pretrained=True)
        
        # replace the classifier with a fully connected layer
        #self.vgg16.classifier = nn.Linear(in_features=25088, out_features=embed_size, bias=True)
        self.resnet50.fc = nn.Linear(in_features=2048, out_features=embed_size, bias=True)
        
        # add another fully connected layer
        self.fc = nn.Linear(in_features=embed_size, out_features=embed_size)
        
        # dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        
        # get the embeddings from vgg16
        x = self.resnet50(x)
        x = F.relu(x)
        
        # pass through dropout
        x = self.dropout(x)
        
        # pass through the fully connected
        x = self.fc(x)
        
        return x

In [68]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        
        # define the properties
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
    def forward(self, embedding, caption_embed):
        
        outputs = torch.empty((batch_size , caption_embed.size(1) , vocab_size))
        hidden_state = torch.zeros((batch_size, hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, hidden_size)).to(device)
        
        # pass the caption word by word
        for t in range(caption_embed.size(1)):

            # for the first time step the input is the embedding vector
            if t == 0:
                hidden_state , cell_state = self.lstm_cell(embedding , (hidden_state, cell_state))
                
            #fot t>0 , input is previous hidden state and cell states
            else :
                hidden_state, cell_state = self.lstm_cell(hidden_state, (hidden_state, cell_state))
            
            #Pass through output layer to match vocab size
            out = self.fc_out(hidden_state)
            
            # build the output tensor
            outputs[:, t, :] = out
    
        return outputs

In [69]:
encoder_rnn = EncoderCNN(embed_size = embed_size)
decoder_rnn = DecoderRNN(embed_size = embed_size , hidden_size = embed_size , vocab_size = vocab_size)

In [70]:
if torch.cuda.is_available():
    print('GPU available')
    device = torch.device("cuda")
else :
    device = torch.device("cpu")
    
device = torch.device("cpu")

GPU available


In [72]:
epochs = 1
ctr = 0
print_every = 1
batch_size = 1
total_records = caption_data.shape[0]

learning_rate = 0.001
optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr = learning_rate)
loss_fn = F.mse_loss

encoder_rnn.to(device)
decoder_rnn.to(device)

train_losses = []
val_losses = []

for epoch in range(epochs):
    
    for df_idx in caption_data.head().index:
        file_name = caption_data.iloc[df_idx].filename.split('#')[0]
        #print(file_name)
        
        image = image_dict[file_name]
        image = image.unsqueeze(0).to(device)
        
        optimizer.zero_grad()
        embedding = encoder_rnn(image)
    
        caption = caption_data.iloc[0].caption
        caption = caption.lower().split(' ')
        
        # init the hidden and cell states to zeros
        caption_embed = torch.zeros((batch_size ,len(caption) , vocab_size) ).to(device)
        
        idx = 0
        batch = 0
        for words in caption:
            word_idx = word_to_idx_dict[words]
            caption_embed[batch][idx][word_idx] = 1
            idx+=1
        
        y_pred = decoder_rnn(embedding = embedding , caption_embed = caption_embed)
        loss = loss_fn(y_pred , caption_embed) 
    
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if ctr % print_every == 0:
            print('epoch : {} \t record number : {} / {} \t train loss : {}'.format(epoch, ctr ,total_records,loss.item()))
            
        ctr +=1

epoch : 0 	 record number : 0 / 40460 	 train loss : 0.00017897880752570927
epoch : 0 	 record number : 1 / 40460 	 train loss : 0.00014989908959250897
epoch : 0 	 record number : 2 / 40460 	 train loss : 0.00012928545766044408
epoch : 0 	 record number : 3 / 40460 	 train loss : 0.00011706171790137887
epoch : 0 	 record number : 4 / 40460 	 train loss : 0.00011699085735017434
